In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re
%matplotlib inline

In [2]:
r = requests.get("https://eda.ru/recepty/russkaya-kuhnya")
r.ok

True

In [12]:
soup = BeautifulSoup(r.content, "html.parser",from_encoding='utf-8')

In [42]:
str = soup.find("span", attrs = {"class":"tag-selector__recipe-count js-updated-page__recipeCount"}).text.split(" ")[0]
int(str)

4719

In [2]:
def get_info_of_rec(obj, cuisine):
    name = obj.find("h3", attrs = {"class":"horizontal-tile__item-title item-title"}).text.strip()
    name = name.replace(u'\xa0', u' ')
    ingr = obj.findAll("span", attrs="js-tooltip js-tooltip-ingredient")
    for key, val in enumerate(ingr):
        ingr[key] = val.text.strip()
    rep = obj.find("span", attrs={"class":"js-bookmark__counter"}).text.strip()
    like = obj.find("span", attrs={"class":"widget-list__like-count"}).text.strip()
    dislike = obj.find("span", attrs={"class":"widget-list__like-count widget-list__like-count_dislike"}).text.strip()
    rating = int(like) - int(dislike) + int(rep)
    elem = {"name":name, "cuisine":cuisine, "ingredients":ingr, "rating":rating}
    return elem

In [3]:
def get_info_of_page(soup,cuisine):
    data = list()
    objects = soup.findAll("div", attrs = {"class":'tile-list__horizontal-tile horizontal-tile js-portions-count-parent js-bookmark__obj'})
    for obj in objects:
        data.append(get_info_of_rec(obj,cuisine))
    return data

In [30]:
from user_agent import generate_user_agent
from time import sleep
import numpy as np

proxies_list = [{'http':'http://'+i} for i in ['67.149.217.254:10200',
                '64.20.74.24:45554','62.37.237.101:8080',
                '180.234.206.77:8080',
                '78.11.85.13:8080','109.188.81.101:8080',
                '139.59.17.113:8080','191.179.147.46:11421',
                '111.68.99.42:8080','80.241.219.66:3128',
                '201.20.94.106:8080','216.229.120.173:45554',
                '116.58.247.31:3128','103.9.115.142:3128',
                '82.164.99.193:10200','80.188.79.138:8080',
                '36.75.113.224:8080',
                '1.20.204.163:8080','97.77.49.151:45554',
                '178.54.44.24:8080',
                '65.182.136.153:45554', '111.76.129.223:808',
                 '203.142.81.205'+':'+'8080', 
                '42.202.35.185'+':'+'8118', '189.16.249.114'+':'+'8080',
                '66.162.122.24'+':'+'8080']]


def get_request(page_num, cuisine):
    sleep(np.random.random())
    url = 'https://eda.ru/recepty/'+cuisine+'-kuhnya?page=' + str(page_num)
    return requests.get(url, headers={'User-Agent': generate_user_agent()}, 
                        proxies=proxies_list[np.random.randint(len(proxies_list))])

In [31]:
def cuisine_parsing(num_of_pages, cuisine):
    all_data = list()
    for i in range(1,num_of_pages+1,1):
        r = get_request(i,cuisine)
        if not r.ok:
            print(error)
            print(i)
        else:
            soup = BeautifulSoup(r.content, "html.parser",from_encoding='utf-8')
            data = get_info_of_page(soup,cuisine)
            all_data += data
    return all_data


In [32]:
name_cuisines = ['kitayskaya','meksikanskaya','gruzinskaya','francuzskaya','yaponskaya','indiyskaya','russkaya','sredizemnomorskaya','uzbekskaya','armyanskaya','tayskaya','italyanskaya','ispanskaya','azerbaydzhanskaya']

In [34]:
our_all_data = list()
for i in name_cuisines:
    our_all_data.append(cuisine_parsing(50,i))

In [36]:
import pickle
output = open('cuisine_dataset.pkl', 'wb')
pickle.dump(our_all_data, output)
output.close()

In [41]:
our_new_all_data = list()
for i in our_all_data:
    our_new_all_data += i 

In [44]:
import pandas as pd
import random
random.shuffle(our_new_all_data)
df = pd.DataFrame(our_new_all_data)
df.head()


,cuisine,ingredients,name,rating
0,francuzskaya,"[Яблоко, Финики, Мед, Лимонный сок, Ваниль, Со...",Коблер,343
1,kitayskaya,"[Филе куриной грудки, Апельсины, Соевый соус, ...",Куриные кусочки в апельсиновом соусе с чили,13
2,kitayskaya,"[Белое сухое вино, Соевый соус, Чеснок, Жидкий...",Курица в кисло-сладком маринаде,66
3,armyanskaya,"[Вода, Пшеничная мука, Соль, Сливочное масло, ...",Армянский лаваш,569
4,kitayskaya,"[Стебель сельдерея, Зеленый лук перья, Соевый ...",Салат из сельдерея по‑китайски,62


In [46]:
with open('cook_dataset.json', 'w') as outfile:  
    json.dump(our_new_all_data, outfile)

In [47]:
with open('cook_dataset.json') as json_file:  
    data = json.load(json_file)
data

[{'name': 'Коблер',
  'cuisine': 'francuzskaya',
  'ingredients': ['Яблоко',
   'Финики',
   'Мед',
   'Лимонный сок',
   'Ваниль',
   'Соль',
   'Молотый имбирь',
   'Кешью',
   'Пекан',
   'Кокосовое масло'],
  'rating': 343},
 {'name': 'Куриные кусочки в апельсиновом соусе с чили',
  'cuisine': 'kitayskaya',
  'ingredients': ['Филе куриной грудки',
   'Апельсины',
   'Соевый соус',
   'Красный перец чили',
   'Чеснок',
   'Перец чили хлопьями',
   'Молотая корица',
   'Лайм'],
  'rating': 13},
 {'name': 'Курица в кисло-сладком маринаде',
  'cuisine': 'kitayskaya',
  'ingredients': ['Белое сухое вино',
   'Соевый соус',
   'Чеснок',
   'Жидкий мед',
   'Тертый имбирь',
   'Лимон',
   'Апельсины',
   'Соль',
   'Перец черный молотый',
   'Растительное масло',
   'Курица'],
  'rating': 66},
 {'name': 'Армянский лаваш',
  'cuisine': 'armyanskaya',
  'ingredients': ['Вода',
   'Пшеничная мука',
   'Соль',
   'Сливочное масло',
   'Сухие дрожжи'],
  'rating': 569},
 {'name': 'Салат из сел

In [48]:
len(data)

6104

In [49]:
df = pd.DataFrame(data)
df.head()

,cuisine,ingredients,name,rating
0,francuzskaya,"[Яблоко, Финики, Мед, Лимонный сок, Ваниль, Со...",Коблер,343
1,kitayskaya,"[Филе куриной грудки, Апельсины, Соевый соус, ...",Куриные кусочки в апельсиновом соусе с чили,13
2,kitayskaya,"[Белое сухое вино, Соевый соус, Чеснок, Жидкий...",Курица в кисло-сладком маринаде,66
3,armyanskaya,"[Вода, Пшеничная мука, Соль, Сливочное масло, ...",Армянский лаваш,569
4,kitayskaya,"[Стебель сельдерея, Зеленый лук перья, Соевый ...",Салат из сельдерея по‑китайски,62
